# Criando o sistema de recomendação - Quantum Finance

In [5]:
import pandas as pd
from math import sqrt
import random

import mysql.connector
from sqlalchemy import create_engine

In [2]:
username = 'root'
password = 'rootpassword'
host = 'localhost' 
database = 'db_recommendations_system'

engine = create_engine(f'mysql+mysqlconnector://{username}:{password}@{host}/{database}')

In [83]:
query = 'SELECT * FROM vw_cliente_produto;'
cliente_produto = pd.read_sql_query(query, engine)
cliente_produto.set_index('id_cliente', inplace=True)

cliente_produto.head()

,customer_name,id_produto,tipo_investimento
id_cliente,,,
1,Guglielma Ripper,2,CDB
1,Guglielma Ripper,3,RDB
1,Guglielma Ripper,9,ETFs
1,Guglielma Ripper,4,LCI/LCA
1,Guglielma Ripper,10,Previdência Privada


In [114]:
query = 'SELECT * FROM quantumfinance_clientes;'
clientes = pd.read_sql_query(query, engine)
clientes.set_index('id_cliente', inplace=True)

clientes.head()

,customer_name,state,age,gender,salary,total_invested
id_cliente,,,,,,
1,Guglielma Ripper,SC,22,M,23036.64,27801.45
2,Linc Mainson,TO,27,M,16192.56,32110.58
3,Loren Birwhistle,RJ,56,M,16053.01,34390.72
4,Clarie Robinet,PE,70,M,45925.22,81220.68
5,Brenden Josling,SE,67,F,24378.72,150108.87


In [206]:
query = 'SELECT * FROM openfinance_produtos;'
produtos = pd.read_sql_query(query, engine)
produtos.set_index('id_produto', inplace=True)

produtos.head()

,tipo_investimento,risco,descricao
id_produto,,,
1,Poupança,Baixo risco,"Rendimento moderado, abaixo da inflação em mui..."
2,CDB,Baixo para médio risco,"Rendimento superior à poupança, varia conforme..."
3,RDB,Baixo para médio risco,"Semelhante ao CDB, com liquidez diária em algu..."
4,LCI/LCA,Baixo risco,"Isento de imposto de renda para pessoa física,..."
5,Fundos de Investimento,Variável,"Depende do tipo do fundo (renda fixa, ações, e..."


### Criando a função de similaridade entre clientes

In [163]:
id_cliente = 8

cliente1 = cliente_produto.loc[id_cliente]
print('Produtos adquiridos nos bancos:', len(cliente1))
cliente1

Produtos adquiridos nos bancos: 4


,customer_name,id_produto,tipo_investimento
id_cliente,,,
8,Averil Vauter,5,Fundos de Investimento
8,Averil Vauter,2,CDB
8,Averil Vauter,10,Previdência Privada
8,Averil Vauter,4,LCI/LCA


In [153]:
cliente2 = cliente_produto.loc[10]
print('Produtos adquiridos nos bancos:', len(cliente2))
cliente2

Produtos adquiridos nos bancos: 6


,customer_name,id_produto,tipo_investimento
id_cliente,,,
10,Stillmann Clarkson,10,Previdência Privada
10,Stillmann Clarkson,1,Poupança
10,Stillmann Clarkson,8,Fundos DI
10,Stillmann Clarkson,2,CDB
10,Stillmann Clarkson,6,Fundos de Renda Fixa
10,Stillmann Clarkson,5,Fundos de Investimento


In [188]:
qtd_itens_totais = len(cliente1['id_produto'].values)

itens_em_comum = [item for item in cliente1['id_produto'].values if item in cliente2['id_produto'].values]
qtd_itens_em_comum = len(itens_em_comum)
similaridade = qtd_itens_em_comum / qtd_itens_totais

print('Similaridade:', similaridade)
print('Itens em comum', itens_em_comum)

Similaridade: 0.25
Itens em comum [10]


### Se o cliente 1 tem 30% dos mesmos produtos que o cliente 2, talvez eles não sejam tão similares, buscamos clientes entre 80% e 90% de similaridade

In [195]:
def obter_similaridade(cliente1, cliente2):
    try:
        qtd_itens_totais = len(cliente1['id_produto'].values)

        itens_em_comum = [item for item in cliente1['id_produto'].values if item in cliente2['id_produto'].values]
        qtd_itens_em_comum = len(itens_em_comum)
        similaridade = qtd_itens_em_comum / qtd_itens_totais

        return similaridade
    except:
        return 0.01

def obter_similares(id_cliente):
    cliente1 = cliente_produto.loc[id_cliente]
    comparar = [id_cliente_2 for id_cliente_2 in cliente_produto.index.unique() if id_cliente_2 != id_cliente]

    listaSimilaridade = [obter_similaridade(cliente1, cliente_produto.loc[id_cliente_2]) for id_cliente_2 in comparar]
    return listaSimilaridade    

In [196]:
meus_similares = clientes.copy()

similaridades = [1]
for i in obter_similares(id_cliente):
    similaridades.append(i)

meus_similares['similaridade'] = similaridades
meus_similares.head()

,customer_name,state,age,gender,salary,total_invested,similaridade
id_cliente,,,,,,,
1,Guglielma Ripper,SC,22,M,23036.64,27801.45,1.00
2,Linc Mainson,TO,27,M,16192.56,32110.58,1.00
3,Loren Birwhistle,RJ,56,M,16053.01,34390.72,1.00
4,Clarie Robinet,PE,70,M,45925.22,81220.68,0.75
5,Brenden Josling,SE,67,F,24378.72,150108.87,1.00


### Criando função para sugestão de produto

In [197]:
# Obter um top 10 de todos com mais de 60% de similaridade

top_10 = meus_similares \
    .query("similaridade >= 0.6 and similaridade < 1") \
    .sort_values(by='similaridade', ascending=False)[1:] \
    .head(10)

top_10

,customer_name,state,age,gender,salary,total_invested,similaridade
id_cliente,,,,,,,
653,Shani Gould,RO,61,F,6317.21,147885.15,0.75
629,Gigi Klimkowski,BA,20,F,8626.88,88460.03,0.75
632,Griz Fenne,GO,56,M,26770.81,60831.13,0.75
634,Hamel Echallier,AL,52,F,32183.91,48366.65,0.75
636,Jenine Porch,RJ,22,F,19736.64,38575.94,0.75
643,Tye Monkton,RJ,26,M,33071.53,147258.86,0.75
644,Cindi Corley,SC,26,F,14533.88,22090.26,0.75
645,Corine Wilstead,GO,67,F,1483.78,119568.13,0.75
647,Candra Feaviour,RN,63,F,1383.31,44566.04,0.75


In [198]:
cliente_similar_aleatório = top_10.reset_index().iloc[random.randint(1, 10)]
cliente_similar_aleatório

id_cliente                 636
customer_name     Jenine Porch
state                       RJ
age                         22
gender                       F
salary                19736.64
total_invested        38575.94
similaridade              0.75
Name: 4, dtype: object

In [225]:
# Sugestão de produto

cliente1 = cliente_produto.loc[id_cliente]

id_cliente_2 = cliente_similar_aleatório['id_cliente']
cliente2 = cliente_produto.query("id_cliente == @id_cliente_2")

produtos_sugeridos = [item for item in cliente2['id_produto'].values if item not in cliente1['id_produto'].values]
produto_escolhido = produtos_sugeridos[random.randint(0, len(produtos_sugeridos)-1)]
produto = produtos.iloc[produto_escolhido]

print('Produtos em potencial:', produtos_sugeridos)
print('Produto sugerido:', produto['tipo_investimento'])

Produtos em potencial: [1, 3, 8, 6, 9]
Produto sugerido: Fundos de Ações
